<a href="https://colab.research.google.com/github/Abhirrami/sdc/blob/main/RESUME_ANALYSER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install -q transformers gradio langchain langchain-community

# Imports
from transformers import pipeline
import gradio as gr

# Define a Hugging Face pipeline for Named Entity Recognition (NER) to extract skills/keywords
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", tokenizer="dbmdz/bert-large-cased-finetuned-conll03-english")

# Function to analyze and extract key information from resume text
def analyze_resume(resume_text):
    # Extract Named Entities (e.g., skills, locations, organization names)
    entities = ner_pipeline(resume_text)
    skills = [entity['word'] for entity in entities if entity['entity'] == 'I-ORG' or entity['entity'] == 'I-LOC'] # organization or location
    important_info = {
        "Skills/Keywords": skills,
        "Word Count": len(resume_text.split()),
        "Suggestions": []
    }

    # Suggest improvements
    if important_info["Word Count"] < 300:
        important_info["Suggestions"].append("Consider expanding the content, as resumes under 300 words may appear incomplete.")

    if "Python" not in skills:
        important_info["Suggestions"].append("Consider adding Python or other relevant technical skills to improve your chances in tech roles.")

    if "Bachelor's Degree" not in resume_text and "Master's Degree" not in resume_text:
        important_info["Suggestions"].append("Make sure to include your educational qualifications, especially your degree.")

    # Return the extracted skills and suggestions
    return important_info

# Define Gradio interface
def resume_interface(resume_text):
    analysis = analyze_resume(resume_text)
    suggestions = "\n".join(analysis["Suggestions"]) if analysis["Suggestions"] else "Your resume looks great!"
    return f"Skills/Keywords: {', '.join(analysis['Skills/Keywords'])}\n\nSuggestions:\n{suggestions}"

# Create Gradio UI
gr.Interface(
    fn=resume_interface,
    inputs=gr.Textbox(lines=15, label="Enter Resume Text Here"),
    outputs=gr.Textbox(label="Analysis and Suggestions"),
    title="Resume Analyzer",
    description="Analyze your resume by entering the text. Get AI-powered insights and suggestions to improve it!"
).launch()
